# **House Price Prediction Regression Problem**

In [1]:
# install
!pip install Pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for Pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=4235c30a5f414487df62008e757a4160149460db441e5cbcfbdcecb1cb9f61b4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built Pyspark


In [2]:
# start spark session
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/22 09:27:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [8]:
# read external data with url
import pandas as pd
house = pd.read_csv('/kaggle/input/house-price-prediction/Boston.csv')
house=spark.createDataFrame(house)

In [9]:
house.show()

+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|              CRIM|  ZN|INDUS|CHAS|   NX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|
+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|           0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296.0|   15.3| 396.9| 4.98|24.0|
|           0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242.0|   17.8| 396.9| 9.14|21.6|
|           0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242.0|   17.8|392.83| 4.03|34.7|
|0.0323699999999999| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222.0|   18.7|394.63| 2.94|33.4|
|           0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222.0|   18.7| 396.9| 5.33|36.2|
|           0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222.0|   18.7|394.12| 5.21|28.7|
|           0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311.0|   15.2| 395.6|12.43|22.9|
|           0.14455|

In [10]:
house.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: long (nullable = true)
 |-- NX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: long (nullable = true)
 |-- TAX: double (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MEDV: double (nullable = true)



In [11]:
house.columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT',
 'MEDV']

In [12]:
# create vector assembler
from pyspark.ml.feature import VectorAssembler 

In [13]:
featureassembler= VectorAssembler(inputCols=['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT'], outputCol='Features')

In [14]:
output = featureassembler.transform(house)

In [15]:
output.show()

24/07/22 09:32:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+--------------------+
|              CRIM|  ZN|INDUS|CHAS|   NX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|            Features|
+------------------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+--------------------+
|           0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296.0|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|           0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242.0|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|           0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242.0|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.0323699999999999| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222.0|   18.7|394.63| 2.94|33.4|[0.03236999999999...|
|           0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222.0|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|           0.02985| 0.0| 2.18|   0|0.458| 6.43|

In [16]:
# create model data
modeldata=output.select('Features','Medv')

In [17]:
modeldata.show()

+--------------------+----+
|            Features|Medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03236999999999...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796000000000...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



In [18]:
# split data
train_data,test_data=modeldata.randomSplit([0.8,0.2])

In [19]:
train_data.show()

+--------------------+----+
|            Features|Medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.01311,90.0,1.2...|35.4|
|[0.0136,75.0,4.0,...|18.9|
|[0.01432,100.0,1....|31.6|
|[0.01951,17.5,1.3...|33.0|
|[0.02055,85.0,0.7...|24.7|
|[0.02729,0.0,7.07...|34.7|
|[0.02731,0.0,7.07...|21.6|
|[0.02763,75.0,2.9...|30.8|
|[0.02875,28.0,15....|25.0|
|[0.03358999999999...|34.9|
|[0.03551,25.0,4.8...|22.9|
|[0.03659,25.0,4.8...|24.8|
|[0.03932,0.0,3.41...|22.0|
|[0.04113,25.0,4.8...|28.0|
|[0.04203,28.0,15....|22.9|
|[0.04378999999999...|19.4|
|[0.04462,25.0,4.8...|23.9|
|[0.04684,0.0,3.41...|22.6|
|[0.04981,21.0,5.6...|23.4|
+--------------------+----+
only showing top 20 rows



In [20]:
# regression model
from pyspark.ml.regression import LinearRegression

In [21]:
reg = LinearRegression(featuresCol='Features', labelCol='Medv')

In [22]:
reg=reg.fit(train_data)

24/07/22 09:32:47 WARN Instrumentation: [c6131640] regParam is zero, which might cause numerical instability and overfitting.
24/07/22 09:32:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/07/22 09:32:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [23]:
reg.coefficients

DenseVector([-0.1239, 0.0548, 0.0764, 2.1266, -18.6708, 3.8741, 0.0044, -1.4759, 0.3766, -0.0161, -0.9175, 0.0112, -0.56])

In [24]:
reg.intercept

35.621513111612664

In [25]:
# prediction
y_pred=reg.evaluate(test_data)

In [26]:
y_pred.predictions.show()

+--------------------+----+------------------+
|            Features|Medv|        prediction|
+--------------------+----+------------------+
|[0.02985,0.0,2.18...|28.7| 25.35835919157061|
|[0.03236999999999...|33.4| 28.77876865211495|
|[0.03584,80.0,3.3...|23.5|30.658917299855183|
|[0.04294,28.0,15....|20.6|27.832681026865167|
|[0.04337,21.0,5.6...|20.5| 24.23003694023781|
|[0.0566,0.0,3.41,...|23.6|30.668554213345708|
|[0.06417,0.0,5.96...|18.9| 23.88534950488541|
|[0.06899,0.0,25.6...|22.0|  22.9619728260934|
|[0.07165,0.0,25.6...|20.3|23.560194514324344|
|[0.08014,0.0,5.96...|21.0|23.106976800672307|
|[0.09164,0.0,10.8...|22.8|24.726592736326744|
|[0.09744,0.0,5.96...|20.0| 22.28285860345421|
|[0.09849,0.0,25.6...|18.8| 21.57161559263733|
|[0.12204,0.0,2.89...|28.4| 28.36716755783614|
|[0.12329,0.0,10.0...|18.8|20.325406470181846|
|[0.14231,0.0,10.0...|18.5|24.904904628120626|
|[0.14866,0.0,8.56...|27.5|24.538538898168795|
|[0.14932,25.0,5.1...|18.7|18.209236777531984|
|[0.16902,0.0

In [27]:
y_pred.meanAbsoluteError

3.332457894840231

In [28]:
y_pred.r2

0.6235549391806119

In [29]:
# close connection to spark only at the end
spark.stop()